In [2]:
import pandas as pd
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 200)

import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import datetime

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# demo 

In [ ]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/00.data/"
infile = filedir + '/ir_icu_basicinfo.csv'

df_demo = pd.read_csv(infile)
df_demo = df_demo.loc[:, ['stay_id', 'age', 'weight', 'gender', 'height', 'ethnicity', 'type1_diabetes',
       'type2_diabetes']]

eth_map = {"UNABLE TO OBTAIN":"UNKNOWN", 'HISPANIC/LATINO':'HISPANIC', 'WHITE':'WHITE', 'BLACK/AFRICAN AMERICAN':'BLACK',
 'UNKNOWN': 'UNKNOWN', 'ASIAN':'ASIAN', 'OTHER':'OTHER', 'AMERICAN INDIAN/ALASKA NATIVE':'AMERICAN_INDIAN'}

df_demo["ethnicity"] = df_demo["ethnicity"].map(eth_map)
df_demo["gender"] = pd.Categorical(df_demo["gender"]).codes
df_demo["ethnicity"] = pd.Categorical(df_demo["ethnicity"]).codes

print(df_demo.gender.unique())
print(df_demo.ethnicity.unique())
print(df_demo.head(2))

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterimp = IterativeImputer(random_state = 123)
df_demo_arr = iterimp.fit_transform(df_demo)
df_demo2 = pd.DataFrame(df_demo_arr)
df_demo2.index = df_demo.index
df_demo2.columns = df_demo.columns
df_demo = df_demo2
df_demo.isna().sum()

df_demo.to_csv("S_basicinfo.csv", index=False)

## vital_lab

### 导入数据

In [2]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/00.data/"
infile = filedir + '/ir_icu_allfeature_Foutlier.csv'

df_fea = pd.read_csv(infile)
df_fea["charttime"] = pd.to_datetime(df_fea["charttime"])#, format='%d/%m/%Y %H:%M:%S')
df_fea = df_fea.drop_duplicates()
df_fea = df_fea.drop(["subject_id","hadm_id", "troponin_i","rdwsd"], axis=1)
print(df_fea.shape)
df_fea.head(2)

(2328549, 88)


,stay_id,charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,spo2,glucose,gcs_motor,gcs_verbal,gcs_eyes,gcs,gcs_unable,urineoutput,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate,chloride,ck_mb,hematocrit,hemoglobin,potassium,sodium,wbc,free_calcium,calcium,totalco2,carboxyhemoglobin,methemoglobin,lactate,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,basophils,eosinophils,lymphocytes,monocytes,neutrophils,atypical_lymphocytes,bands,immature_granulocytes,metamyelocytes,nrbc,troponin_t,albumin,globulin,total_protein,aniongap,bun,creatinine,d_dimer,fibrinogen,thrombin,inr,pt,ptt,mch,mchc,mcv,platelet,rbc,rdw,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ggt,ld_ldh,crp
0,39060235,2160-05-18 10:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,421.0,45.0,NaN,NaN,NaN,NaN,NaN,7.35,0.0,NaN,104.0,NaN,36.0,11.9,3.4,137.0,NaN,1.17,NaN,26.0,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39060235,2160-05-18 11:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,462.0,41.0,NaN,NaN,NaN,NaN,NaN,7.40,0.0,NaN,103.0,NaN,36.0,12.0,3.7,138.0,NaN,1.18,NaN,26.0,NaN,NaN,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 去除没有一个值的变量

In [3]:
'''df_fea_stat = df_fea.isnull().sum().reset_index()
df_fea_stat.columns = ["index", "na_num"]
df_fea_stat["%"] = df_fea_stat["na_num"]/df_fea.shape[0]
print(df_fea_stat.sort_values(by="%",ascending=False).head(2))

print("na %=1","troponin_i","rdwsd")
df_fea = df_fea.drop(["troponin_i","rdwsd"], axis=1)
print(df_fea.shape)
'''

         index   na_num    %
60  troponin_i  2328549  1.0
80       rdwsd  2328549  1.0
na %=1 troponin_i rdwsd
(2328549, 90)


### 获取处理成窗口形式的特征数据 = 将点数据mapping到60 min窗口中

In [18]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/01.action/"
infile = filedir + 'A_IR_win60.csv'

df_ac = pd.read_csv(infile)
df_ac["starttime"] = pd.to_datetime(df_ac["starttime"])
df_ac["endtime"] = pd.to_datetime(df_ac["endtime"])
df_ac = df_ac.sort_values(by=["stay_id", "starttime"])
df_ac = df_ac.loc[:,['stay_id', 'starttime', 'endtime',"traj_id"]]

df_fea = df_fea[df_fea["stay_id"].isin(df_ac.stay_id.unique().tolist())] 

df_fea_add = pd.merge( df_fea, df_ac, how="inner", on = "stay_id")
df_fea_add = df_fea_add[(df_fea_add["starttime"] <= df_fea_add["charttime"]) & (df_fea_add["charttime"] <= df_fea_add["endtime"])]
#df_fea_add.to_csv("df_fea.csv",index=False)

### 插补
- 前向
- MI

In [20]:
df_fea_add = pd.read_csv("df_fea.csv")
df_fea_add["starttime"] = pd.to_datetime(df_fea_add["starttime"])
df_fea_add["endtime"] = pd.to_datetime(df_fea_add["endtime"])
df_fea_add["charttime"] = pd.to_datetime(df_fea_add["charttime"])

df_im = pd.DataFrame()
for i,j in df_fea_add.groupby(by=["traj_id"]):
    j = j.sort_values("charttime")
    j = j.fillna(method='ffill', limit=12) 
    df_im = pd.concat([df_im, j], axis = 0)
df_im.to_csv("S_fea_win60_im.csv",index=False)

### 删除缺失值大于80%的变量

In [13]:
df_im = pd.read_csv("S_fea_win60_im.csv")

#缺失统计
df_fea_stat = df_im.isnull().sum().reset_index()
df_fea_stat.columns = ["index", "na_num"]
df_fea_stat["%"] = df_fea_stat["na_num"]/df_im.shape[0]
tmp_lst = df_fea_stat[df_fea_stat["%"] <= 0.80]["index"].tolist()

df_fea_stat.sort_values(by="%",ascending=False).head(100)

df_im = df_im[tmp_lst]
df_im =df_im.drop(["charttime"],axis=1)
df_im.to_csv("tmp_S_fea_win60_im2.csv",index=False)

### MI

In [16]:
df_im = pd.read_csv("tmp_S_fea_win60_im2.csv")
df_im.replace([np.inf, -np.inf], np.nan, inplace=True)
df_bind = df_im.loc[:,["stay_id", "starttime","endtime","traj_id"]]
df_im = df_im.drop(["stay_id", "starttime","endtime","traj_id"],axis=1)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterimp = IterativeImputer(random_state = 123)
df_im_arr = iterimp.fit_transform(df_im)
df_im1 = pd.DataFrame(df_im_arr)
df_im1.index = df_im.index
df_im1.columns = df_im.columns
#df_im1.to_csv("S_fea_win60_im2.csv",index=False)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


### 输出仅有每小时的最大，最小，中值


In [30]:
df_fea_sar = pd.read_csv("S_fea_win60_im2.csv")
df_fea_sar = pd.concat([df_fea_sar,df_bind ],axis=1)

# 输出仅有每小时的最大，最小，均值
df_grouped = df_fea_sar.groupby(["stay_id", 'starttime', 'endtime' ,"traj_id"]).agg(['max', 'min', 'mean'])
df_grouped.columns = ['_'.join(col) for col in df_grouped.columns]
df_grouped = df_grouped.reset_index()
#df_grouped.to_csv("S_fea_win60.csv", index=False)

# get win-size fluid, dex

## fluid, dex data load

In [33]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/00.data/"

infile = filedir + '/ir_icu_fluid_colloid_and_crystalloid.csv'
df_fluid = pd.read_csv(infile)
df_fluid["starttime"] = pd.to_datetime(df_fluid["starttime"], format='%d/%m/%Y %H:%M:%S')
df_fluid["endtime"] = pd.to_datetime(df_fluid["endtime"], format='%d/%m/%Y %H:%M:%S')
df_fluid = df_fluid.loc[(df_fluid.loc[:, 'endtime'] - df_fluid.loc[:, 'starttime']) > datetime.timedelta(minutes=1),:]

df_fluid = df_fluid.drop(["patientweight"],axis=1)
df_fluid.columns = ['stay_id', 'fluid_starttime', 'fluid_endtime', 'colloid_bolus']
df_fluid.head(2)


,stay_id,fluid_starttime,fluid_endtime,colloid_bolus
0,30000213,2162-06-21 05:40:00,2162-06-21 06:06:00,3
1,30000213,2162-06-21 05:40:00,2162-06-21 12:07:00,6


In [34]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/00.data/"

infile = filedir + '/ir_icu_glucose_infusion.csv'
df_dex = pd.read_csv(infile)
df_dex["starttime"] = pd.to_datetime(df_dex["starttime"], format='%d/%m/%Y %H:%M:%S')
df_dex["endtime"] = pd.to_datetime(df_dex["endtime"], format='%d/%m/%Y %H:%M:%S')
df_dex = df_dex.loc[:,['stay_id', 'starttime', 'endtime','dex_dose']]
df_dex.columns = ['stay_id', 'dex_starttime', 'dex_endtime', 'dex_dose']
df_dex.head(2)

,stay_id,dex_starttime,dex_endtime,dex_dose
0,31851222,2171-03-02 21:14:00,2171-03-02 21:15:00,2.5
1,31851222,2171-03-03 09:00:00,2171-03-03 09:01:00,10.0


In [36]:
filedir = "/public/hanl/jupyter_dir/ir_recommend_v2/01.action/"
infile = filedir + '/A_IR_win60.csv'

df_ac_win = pd.read_csv(infile)
print(df_ac_win.head(1))
df_ac_win["starttime"] = pd.to_datetime(df_ac_win["starttime"])
df_ac_win["endtime"] = pd.to_datetime(df_ac_win["endtime"])
df_ac_win.head(2)

    stay_id            starttime              endtime  amount  rate  interval  \
0  30001148  2156-08-30 19:15:00  2156-08-30 20:15:00     2.0   2.0      60.0   

   traj_id  
0        1  


,stay_id,starttime,endtime,amount,rate,interval,traj_id
0,30001148,2156-08-30 19:15:00,2156-08-30 20:15:00,2.0,2.0,60.0,1
1,30001148,2156-08-30 20:15:00,2156-08-30 21:15:00,2.0,2.0,60.0,1


## win-size fluid

In [37]:
df_ac_fluid = pd.merge(df_ac_win, df_fluid, on=['stay_id'], how='inner')
df_merge = df_ac_fluid

tem = df_merge.loc[:, ['endtime', 'fluid_endtime']].min(axis=1) - df_merge.loc[:, ['starttime', 'fluid_starttime']].max(axis=1)
tem= tem / (df_merge.loc[:, 'endtime'] - df_merge.loc[:, 'starttime'])

colloid_bolus_win = tem.clip(0, tem) * df_merge.loc[:, 'colloid_bolus']
df_merge.loc[:,'colloid_bolus_win'] = colloid_bolus_win

df_merge = df_merge.drop(['fluid_starttime','fluid_endtime', 'colloid_bolus',], axis =1)
df_ac_win_addfluid = df_merge.groupby(['stay_id', 'starttime', 'endtime', 'amount', 'rate'], as_index=False).agg(sum)

## win-size dex

In [39]:
df_ac_dex = pd.merge(df_ac_win, df_dex, on=['stay_id'], how='inner')
df_merge = df_ac_dex

tem = df_merge.loc[:, ['endtime', 'dex_endtime']].min(axis=1) - df_merge.loc[:, ['starttime', 'dex_starttime']].max(axis=1)
tem= tem / (df_merge.loc[:, 'endtime'] - df_merge.loc[:, 'starttime'])


dex_win = tem.clip(0, tem) * df_merge.loc[:, 'dex_dose']
df_merge.loc[:,'dex_win'] = dex_win

df_merge = df_merge.drop(['dex_starttime','dex_endtime', 'dex_dose',], axis =1)
df_ac_win_adddex = df_merge.groupby(['stay_id', 'starttime', 'endtime', 'amount', 'rate'], as_index=False).agg(sum)


In [ ]:
print(df_ac_win.shape)
print(df_ac_win_addfluid.shape)
print(df_ac_win_adddex.shape) # 出现行数不等的原因是，df_dex中有143个病人完全没有dex输入的记录，病人id见下面的lst：
#lst = pd.concat([df_ac_win, df_ac_win_adddex,df_ac_win_adddex]).drop_duplicates(keep=False,subset=['stay_id', 'starttime', 'endtime', 'amount', 'rate']).stay_id.unique()

In [40]:
df_ac_win60_add = pd.merge(df_ac_win_addfluid,df_ac_win_adddex,how="left", left_on=['stay_id', 'starttime', 'endtime', 'amount', 'rate',"traj_id"],right_on=['stay_id', 'starttime', 'endtime', 'amount', 'rate',"traj_id"])
df_ac_win60_add[["dex_win"]] = df_ac_win60_add[["dex_win"]].fillna(value=0)


In [44]:
df_ac_win60_add =df_ac_win60_add.drop(["amount","rate","interval_x","interval_y"],axis=1)
df_ac_win60_add.to_csv("S_bolus_dextrose.csv", index=False)

In [46]:
df_ac_win60_add

,stay_id,starttime,endtime,traj_id,colloid_bolus_win,dex_win
0,30001148,2156-08-30 19:15:00,2156-08-30 20:15:00,9,6.000000,0.0
1,30001148,2156-08-30 20:15:00,2156-08-30 21:15:00,9,256.000000,0.0
2,30001148,2156-08-30 21:00:00,2156-08-30 22:00:00,9,1006.000000,0.0
3,30002415,2126-12-17 11:59:00,2126-12-17 12:59:00,46,380.716667,0.0
4,30002415,2126-12-17 12:59:00,2126-12-17 13:59:00,46,332.000000,0.0
...,...,...,...,...,...,...
266126,39999552,2186-07-18 03:18:00,2186-07-18 04:18:00,431733,256.000000,0.0
266127,39999552,2186-07-18 04:18:00,2186-07-18 05:18:00,431733,262.133333,0.0
266128,39999552,2186-07-18 05:18:00,2186-07-18 06:18:00,431733,263.000000,0.0
266129,39999552,2186-07-18 06:18:00,2186-07-18 07:18:00,431733,384.116667,0.0
